Based on tutorial from https://towardsdatascience.com/pre-trained-word-embedding-for-text-classification-end2end-approach-5fbf5cd8aead

In [1]:
import pickle
import numpy as np
import pandas
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd "/content/drive/My Drive/dis study abroad/Artificial Neural Networks"


Mounted at /content/drive
/content/drive/My Drive/dis study abroad/Artificial Neural Networks


In [3]:
%ls

emily_ingredients_model.ipynb     full_dataset.csv.zip     NLG_cleaning.ipynb
flavor_graph.ipynb                ingredients_model.ipynb  nodes_191120.csv
FlavorGraph_NodeEmbedding.pickle  ner_embeddings.pkl


In [4]:
# open and load data from pickle file
file = open('FlavorGraph_NodeEmbedding.pickle', 'rb')

# keys are strings for node_id, value is embedding
data = pickle.load(file)

file.close()

In [5]:
# load dataframe of node_ids and ingredients
df = pandas.read_csv('nodes_191120.csv')

In [6]:
# just get the embeddings for ingredients
ing_embeddings = {}

for i in range(len(df)):
    if df.loc[i, 'node_type'] == "ingredient":
        # map the name of the ingredient to the embedding
        ing_embeddings[df.loc[i, 'name']] = data[str(df.loc[i, 'node_id'])]
        


In [7]:
! pip install pickle5
import pickle5 as pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 5.2 MB/s 


In [8]:
# open and load data from pickle file
ner_embeddings_file = open('ner_embeddings.pkl', 'rb')

# keys are strings for node_id, value is embedding
ner_embeddings = pickle.load(ner_embeddings_file)

ner_embeddings_file.close()

In [9]:
subset= ner_embeddings.sample(n=1000)

In [10]:
subset

,NER,embeddings,num_ingredients,start_ingredient,embedding_matrix_index_list,start_ingredient_index
530028,"[chicken, frozen_broccoli_spear, cream_of_mush...","[[-0.22679155, 0.100118645, 0.10348936, 0.2072...",6,cream_of_mushroom_soup,"[1066, 2464, 1596, 3845, 3499, 1015]",1596
881719,"[egg, peanut_butter, sugar, vanilla, milk, unb...","[[0.25692862, 0.08700464, 0.23288268, 0.169938...",6,vanilla,"[2052, 4375, 5769, 6206, 3926, 6140]",6206
996228,"[butter, sweet_onion, farmer_cheese, muenster_...","[[-0.4366374, 0.026041824, 0.15680166, 0.11945...",9,muenster_cheese,"[727, 5850, 2118, 4028, 1527, 1508, 5228, 3926...",4028
824109,"[ground_beef, salt, tomato_soup, milk, mozzare...","[[-0.10498587, 0.1431403, 0.041966103, -0.1956...",9,macaroni,"[2864, 5228, 6040, 3926, 4023, 4175, 4417, 374...",3749
260373,"[oil, egg, sugar, flour, baking_soda, baking_p...","[[-0.113179356, -0.10267937, -0.19532534, -0.0...",9,baking_soda,"[4155, 2052, 5769, 2242, 277, 276, 4401, 148, ...",277
...,...,...,...,...,...,...
1806231,"[pine_nut, olive_oil, balsamic_vinegar, lemon_...","[[-0.022968607, -0.17425817, 0.041393567, -0.2...",6,lemon_juice,"[4518, 4171, 281, 3499, 5371, 5228]",3499
1207197,"[quinoa, olive_oil, olive_oil, black_bean, cor...","[[-0.0034792803, 0.17953567, 0.10123219, -0.33...",12,corn,"[4830, 4171, 4171, 440, 1500, 4915, 4417, 2309...",1500
994391,"[chicken, chicken, tomato_sauce, red_wine_vine...","[[-0.22679155, 0.100118645, 0.10348936, 0.2072...",18,red_pepper,"[1066, 1066, 6039, 4989, 4918, 156, 5601, 3479...",4963
497440,"[okra, pepper, garlic, white_vinegar, salt, ce...","[[0.15744048, -0.18594559, -0.41463867, 0.2232...",6,okra,"[4161, 4417, 2611, 6479, 5228, 974]",4161


In [11]:
# # Finally, we pad sequenced data and define predictors and labels. 
# # We use predictors to guess what is the next word in a sequence and labels 
# # to correct the model’s predictions.
from tensorflow.keras.preprocessing.sequence import pad_sequences
# import numpy as np
# import tensorflow.keras.utils as ku

# # pad sequences
# max_sequence_len = max([len(x) for x in input_sequences])
# input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# # create predictors and label
# total_words = len(tokenizer.word_index) + 1
# predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
# label = ku.to_categorical(label, num_classes=total_words)

In [12]:
# Train the model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
import tensorflow as tf

In [13]:
EMBEDDING_DIMENSIONS = 300

def construct_embedding_matrix(embeddings):
    num_ing = len(embeddings)

    # initialize a matrix of zeros
    embedding_matrix = np.zeros((num_ing, EMBEDDING_DIMENSIONS)) # each embedding has 300 dimensions

    next_row = 0

    for i in embeddings:
        v = embeddings.get(i)
        embedding_matrix[next_row] = v
        next_row+=1
    
    return embedding_matrix

In [14]:
embedding_matrix = construct_embedding_matrix(ing_embeddings)

In [15]:
embedding_matrix

array([[-0.10600116,  0.04714949,  0.10841199, ..., -0.03144248,
        -0.06629407, -0.1286629 ],
       [-0.01582931,  0.09736368, -0.00062261, ..., -0.09226537,
        -0.12149926, -0.12204846],
       [-0.10132008,  0.03372396,  0.06472784, ..., -0.22692445,
        -0.04366636, -0.20344618],
       ...,
       [-0.19128327,  0.17544127, -0.09963894, ..., -0.20900002,
        -0.17799097, -0.1547064 ],
       [ 0.02008764,  0.04900858, -0.26409724, ..., -0.19495088,
        -0.16633987, -0.21576235],
       [ 0.20899913, -0.15171458, -0.25460058, ..., -0.18800448,
        -0.08664556, -0.07758268]])

In [16]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [17]:
MAX_SEQUENCE_LENGTH = 20 # this is arbitrary right now

from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D, Dropout, Flatten, Conv1D, MaxPooling1D
from keras.initializers import Constant
from keras.optimizers import Adam

In [18]:
# not sure if the first two parameters are what we need vs. training dirrectly on embeddings (since we're trying to use them as weights)
x_train, x_val, y_train, y_val= train_test_split(subset['start_ingredient_index'], subset['embedding_matrix_index_list'], test_size= 0.2)

In [19]:
#print(type(X_train['start_ingredient']))
import numpy as np

# x_train_array= np.asarray(x_train).astype('float32')
# x_val_array= np.asarray(x_val).astype('float32')

x_train = np.asarray(x_train).astype('float32').reshape((-1,1))
x_val = np.asarray(x_val).astype('float32').reshape((-1,1))

# y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
# y_val = np.asarray(y_val).astype('float32').reshape((-1,1))

y_train = pad_sequences(y_train, maxlen = MAX_SEQUENCE_LENGTH)
y_val = pad_sequences(y_val, maxlen = MAX_SEQUENCE_LENGTH)

In [20]:
print("x_train: ", len(x_train))
print("x_val: ", len(x_val))
print("y_train: ", len(y_train))
print("y_val: ", len(y_val))

x_train:  800
x_val:  200
y_train:  800
y_val:  200


In [21]:
# not sure if these are padded the way we want to? but at least the model fits

print('x_train: ', x_train[0])
print('y_train: ', y_train[0])
print('x_val: ', x_val[0])
print('y_val: ', y_val[0])

x_train:  [5574.]
y_train:  [   0    0    0    0    0    0    0    0    0    0    0    0 5574 3845
  505 4420 2380 2374 3479 5284]
x_val:  [971.]
y_val:  [   0    0    0    0    0    0    0    0 5420 2866 6614 2242 5228  971
 4296 3926 2052 6323  974 1185]


In [22]:
MAX_SEQUENCE_LENGTH

20

In [23]:
model=Sequential()
embedding = Embedding(len(embedding_matrix),
                            EMBEDDING_DIMENSIONS,
                            weights=[embedding_matrix],
                            input_length=1)
model.add(embedding)
model.add(Flatten()) # not sure if this is needed
# model.add(Dropout(0.2))
model.add(Dense(20))
model.add(Activation("tanh"))
model.add(Dense(20))
model.add(Activation("softmax"))

# model.add(Dense(MAX_SEQUENCE_LENGTH, activation='softmax'))
# model.add(Dense(10))
# model.add(Bidirectional(LSTM(150, return_sequences = True)))
# model.add(LSTM(128))
# model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.5))
# model.add(Dense(MAX_SEQUENCE_LENGTH, activation='softmax'))
# # compile the model
# optimzer = Adam(clipvalue=0.5) # clip value to avoid the gradient exploding

# model.compile(optimizer=optimzer, 
#               loss='categorical_crossentropy', 
#               metrics=['acc',f1_m,precision_m, recall_m])

# fit the model
# history = model.fit(predictors, label, epochs=20, verbose=1)
optimzer = Adam(clipvalue=0.5) # clip value to avoid the gradient exploding

# loss functions:
# Sparse cross-entropy addresses this by performing the same cross-entropy calculation of error, without requiring that the target variable be one hot encoded prior to training.

# model.compile('rmsprop', 'mse')
# output_array = model.predict(x_train)
# print(output_array.shape)
# assert output_array.shape == (32, 10, 64)

model.compile(
    loss="categorical_crossentropy", optimizer=optimzer, metrics=["accuracy"]
)
history = model.fit(x_train, y_train, batch_size=100, epochs=20, validation_data=(x_val, y_val), verbose=2)


Epoch 1/20
8/8 - 2s - loss: 72436.5938 - accuracy: 0.1175 - val_loss: 68013.0234 - val_accuracy: 0.2000 - 2s/epoch - 212ms/step
Epoch 2/20
8/8 - 0s - loss: 69031.1484 - accuracy: 0.1600 - val_loss: 65321.4102 - val_accuracy: 0.2450 - 301ms/epoch - 38ms/step
Epoch 3/20
8/8 - 0s - loss: 66456.1562 - accuracy: 0.1725 - val_loss: 63382.3906 - val_accuracy: 0.2550 - 346ms/epoch - 43ms/step
Epoch 4/20
8/8 - 0s - loss: 64685.4609 - accuracy: 0.1825 - val_loss: 62124.6719 - val_accuracy: 0.2550 - 295ms/epoch - 37ms/step
Epoch 5/20
8/8 - 0s - loss: 63620.0234 - accuracy: 0.1887 - val_loss: 61431.0508 - val_accuracy: 0.2550 - 307ms/epoch - 38ms/step
Epoch 6/20
8/8 - 0s - loss: 63107.8516 - accuracy: 0.1875 - val_loss: 61155.0312 - val_accuracy: 0.2550 - 296ms/epoch - 37ms/step
Epoch 7/20
8/8 - 0s - loss: 62975.8555 - accuracy: 0.1875 - val_loss: 61095.3594 - val_accuracy: 0.2550 - 383ms/epoch - 48ms/step
Epoch 8/20
8/8 - 0s - loss: 63045.3438 - accuracy: 0.1813 - val_loss: 61173.2695 - val_accur

In [24]:
output_array = model.predict(x_val)
print(output_array.shape)
output_array
# it's basically converting the list of 20 ingredient indices into supposed embeddings ... 
# but the dimensions aren't 300...something is wrong here


7/7 [==============================] - 0s 3ms/step
(200, 20)


array([[0.0011322 , 0.00172231, 0.01212135, ..., 0.048947  , 0.09629139,
        0.1799517 ],
       [0.00106946, 0.00162091, 0.01186688, ..., 0.04831038, 0.09657963,
        0.17827915],
       [0.00108848, 0.00165179, 0.01194119, ..., 0.0484525 , 0.09660231,
        0.17820072],
       ...,
       [0.00106967, 0.00162125, 0.01186757, ..., 0.04831283, 0.09658513,
        0.17829186],
       [0.00106946, 0.001621  , 0.01186674, ..., 0.04831028, 0.09657727,
        0.17828135],
       [0.00106939, 0.00162071, 0.01186655, ..., 0.04830955, 0.09658073,
        0.17827591]], dtype=float32)

In [32]:
print(x_val[0])
print(output_array[0])
#6355 is water

[971.]
[0.0011322  0.00172231 0.01212135 0.0153873  0.01698954 0.01307626
 0.00290594 0.0082397  0.04675011 0.00584953 0.01092933 0.01124354
 0.20351382 0.03905983 0.07671699 0.08750225 0.12166988 0.048947
 0.09629139 0.1799517 ]


In [26]:
print(x_val[100])
print(output_array[100])
#4633 is pork

[6355.]
[0.00106943 0.00162085 0.0118669  0.01509725 0.01643776 0.01272949
 0.00275526 0.00795874 0.04631847 0.00558949 0.01053515 0.01094191
 0.20790097 0.03884815 0.0772872  0.08679832 0.12307815 0.04831047
 0.09658027 0.17827575]


In [27]:
ner_embeddings

,NER,embeddings,num_ingredients,start_ingredient,embedding_matrix_index_list,start_ingredient_index
1,"[beef, chicken_breast, cream_of_mushroom_soup,...","[[-0.33506337, 0.08803183, -0.24923237, 0.0341...",4,beef,"[339, 1076, 1596, 5574]",339
4,"[peanut_butter, graham_cracker_crumb, butter, ...","[[-0.17900778, -0.039890748, -0.13081385, 0.30...",5,peanut_butter,"[4375, 2733, 727, 4719, 1197]",4375
9,"[pineapple, condensed_milk, lemon, pecan, grah...","[[0.16943195, -0.31285393, -0.23219007, 0.1775...",5,pecan,"[4519, 1412, 3479, 4401, 2735]",4401
12,"[chicken, flour, barbecue_sauce]","[[-0.22679155, 0.100118645, 0.10348936, 0.2072...",3,barbecue_sauce,"[1066, 2242, 297]",297
14,"[pie_filling, pineapple, condensed_milk, lemon...","[[-0.021126581, 0.1452302, -0.09510492, -0.035...",4,pie_filling,"[4487, 4519, 1412, 3499]",4487
...,...,...,...,...,...,...
2231129,"[brown_sugar, sour_cream, butter, butter, corn...","[[-0.37061608, -0.21077241, -0.011841524, -0.1...",14,brown_sugar,"[697, 5574, 727, 727, 1529, 6206, 6228, 2052, ...",697
2231132,"[egg, vegetable_oil, white_sugar, pumpkin_pure...","[[0.25692862, 0.08700464, 0.23288268, 0.169938...",15,ground_ginger,"[2052, 6286, 6473, 4793, 2242, 276, 277, 5228,...",2889
2231135,"[bread, italian_sauce, frozen_meatball]","[[0.17803018, 0.19869211, 0.01149651, 0.133690...",3,italian_sauce,"[624, 3251, 2510]",3251
2231137,"[chocolate_hazelnut_spread, tortilla, butter, ...","[[0.16573386, -0.09754042, -0.16330065, 0.2495...",5,tortilla,"[1229, 6061, 727, 3822, 2998]",6061


In [28]:
# # ignore
# class myCallback(tf.keras.callbacks.Callback):
# 	def on_epoch_end(self, epoch, logs={}):
# 		if(logs.get('accuracy')>0.93):
# 			print("\nReached 93% accuracy so cancelling training!")
# 			self.model.stop_training = True

# callbacks = myCallback()

# history = model.fit(predictors, label, epochs=10, verbose=1, callbacks=[callbacks])


In [29]:
# # ignore
# seed_text = "water"
# seed_text_two = "olive oil"
# seed_text_three = "cinnamon"
# seed_text_three = "garlic"
# seed_text_three = "tequila"

In [30]:
# #ignore
# next_words = 5

# for _ in range(next_words):
#   token_list = tokenizer.texts_to_sequences([seed_text])[0]
#   token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
#   predict_x=model.predict(token_list) 
#   predicted=np.argmax(predict_x,axis=1)
  
#   output_word = ""
#   for word, index in tokenizer.word_index.items():
#     if index == predicted:
#       output_word = word
#       break
#   seed_text += " " + output_word

# seed_text = (seed_text + ".").capitalize()
# print(seed_text)